In [28]:
import pandas as pd
from pyspark.ml.recommendation import ALS
from pyspark import SparkContext as sc
import pyspark

In [4]:
user_card_matrix=pd.read_excel(r'./user_card_matrix_for_rec.xlsx',encoding='utf-16')

In [5]:
user_card_matrix=user_card_matrix.rename(columns={'Unnamed: 0':'user'})

In [6]:
user_card_matrix.head()

,user,三信銀行-VISA白金卡,上海商銀-TeresaCard,上海商銀-小小兵Bello回饋卡,上海商銀-簡單卡,土地銀行-JCB一卡通聯名晶緻卡,中國信託-ANA聯名卡,中國信託-LINE Pay卡,中國信託-MUJI無印良品聯名卡,中國信託-Yahoo聯名卡,...,臺灣銀行-導盲犬認同卡,遠東商銀-Bankee信用卡,遠東商銀-快樂信用卡,聯邦銀行-全國加油聯名卡,聯邦銀行-幸福御守御璽卡,聯邦銀行-法拉利無限卡,聯邦銀行-紅利卡,聯邦銀行-理財型白金卡,聯邦銀行-微風聯名卡,聯邦銀行-賴點一卡通御璽卡
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
colums_list=list(user_card_matrix.columns.values)[1:]

In [8]:
spark_als_data=pd.melt(user_card_matrix, id_vars=['user'], value_vars=colums_list)

In [9]:
len(spark_als_data)

476820

In [31]:
spark_als_data.to_csv(r'./spark_als_data.csv',encoding='utf-8')

In [24]:
spkdata=spark.read.load(r'./spark_als_data.csv')

NameError: name 'spark' is not defined

In [16]:
sparkcontext

NameError: name 'sparkcontext' is not defined

In [29]:
spark

NameError: name 'spark' is not defined

In [ ]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.recommendation import ALS
from pyspark import SparkContext as sc
from pyspark.sql import Row
from pyspark.sql.functions import split, explode

data = sc.textFile("/home/cloudera/PythonProjects/creditcard/spark_als_data.csv") 
datasplit=data.map(lambda l: l.split(',')) 
ratingsRDD = datasplit.map(lambda  p: Row(userId=int(p[1]), cardId=p[2],rating=int(p[3])))
ratings=spark.createDataFrame(ratingsRDD)
indexer = StringIndexer(inputCol="cardId", outputCol="cardIdIndex") 
idxdf=indexer.fit(ratings).transform(ratings)
(training, test) = idxdf.randomSplit([0.8, 0.2])
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="cardIdIndex", 
          ratingCol="rating",coldStartStrategy="drop")
model = als.fit(training)
predictions = model.transform(test)
userRecs = model.recommendForAllUsers(10)

#card idx對照表
card_mapping_df=idxdf.select('CardId','CardIdIndex').distinct()

#篩選最後一筆userID
newuser=userRecs.where('userID==2648')
rec=newuser.select(explode('recommendations')).alias("recommendation").rdd.map(lambda x:(x[0])) 
rec_df=spark.createDataFrame(rec)



# userRecs.select('userID',explode('recommendations')).alias("recommendation").
# select('userID',explode('col'))    
# userRecs.select('userID',explode('recommendations')).alias("recommendation").rdd.map(lambda x:x).take(10)          
# userRecs.select('userID',explode('recommendations')).alias("recommendation").rdd.flatMap(lambda x:(x[1])
#      ...: ).map(lambda x :x).take(10)  
                                                           
card_df_f=card_mapping_df.withColumn("cardIdIndex", card_mapping_df["cardIdIndex"].cast(IntegerType())) 
card_df=card_df_f.alias('card_df') 
user_rec=rec_df.alias('user_rec')  



final_rec_df=user_rec.join(card_df,user_rec.cardIdIndex==card_df.cardIdIndex,how='left')

#排序
final_rec_5=final_rec_df.sort(final_rec_df.rating.desc()) 


#by userid
userRecs.select('userID',explode('recommendations')).alias("recommendation").rdd.map(lambda x:(x[0],x[1]
     ...: )).map(lambda x :x).reduceByKey(lambda x,y:(x,y)).collect() 

In [6]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.recommendation import ALS
from pyspark import SparkContext as sc
from pyspark import SparkConf

In [7]:
sc = sc.getOrCreate(SparkConf().setMaster("local[*]"))
data = sc.textFile("./spark_als_data.csv") 

Exception: Java gateway process exited before sending its port number